In [2]:
import csv
import os
import traceback
import numpy as np
import tqdm
import pandas as pd
import multiprocessing as mp
import gc
from datetime import timedelta, date
import uuid

# import rsa_data_summary as rd
# import rsa_data_wim as wim
# import rsa_headers as rh
import config
import queries as q
import tools

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)

In [3]:
# individual tables for type 30 and 60 test
# files = tools.getfiles(r'C:\FTP\Syntell\SMEC RSA Files_GP PRM Sites_Dec21toFeb22')
test_header = r"C:\PQ410\2 - Data In\TIS Data\CSIR\2010\Secondary Loop\0007.rsa"
file = r"C:\PQ410\2 - Data In\TIS Data\GDRT\GP_RSA 2014\0038.RSA"
t21_file =r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_1.RSA"
t10_file = r"C:\FTP\Syntell\SMEC RSV Files_GP PRM Sites_Dec21toFeb22_individuals\0006-20211231.RSV"
t10_file2 = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"
type60_file = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"
new_test = r"C:\PQ410\2 - Data In\TIS Data\Disks Sept 2020\Data Discs\2017\2017Q2\Historic\GPG CTO 2011 Yearbook\RSA Data\0317.rsa"

In [3]:
# electronic_count_data_type_21_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_21 limit 1",config.ENGINE).columns)
# electronic_count_data_type_30_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_30 limit 1",config.ENGINE).columns)
electronic_count_data_type_60_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_60 limit 1",config.ENGINE).columns)
# electronic_count_data_type_70_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_70 limit 1",config.ENGINE).columns)
# header_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_header limit 1",config.ENGINE).columns)

In [4]:
# df = tools.to_df(t10_file)
dfa = pd.read_csv(t10_file2, header=None) #, sep='\s+', engine='python')
# dfa = pd.read_csv(t21_file, header=None, sep='\s+', engine='python')

dfa = dfa[0].str.split("\s+|,\s+|,|;|;\s+", expand=True)

In [78]:
df = pd.DataFrame(dfa.loc[
    (~dfa[0].isin(["H0", "H9", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
    & ((
        (dfa[0].isin(["21", "22", "70", "30", "31", "13", "60"]))
        & (dfa[1].isin(["0", "1", "2", "3", "4"]))
        & (dfa.loc[dfa[0].isin(["21", "70", "30", "13", "60"])][3].astype(int) > 80)
    ) | (
        (dfa[0].isin(["10"]))
        & (~dfa[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
        & (dfa.loc[dfa[0].isin(["10"])][4].astype(int) > 80)
    ))           
    ]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [79]:
head_df = pd.DataFrame(dfa.loc[
    (dfa[0].isin(["H0", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
    | (
        (dfa[0].isin(["21", "70", "30", "13", "60"]))
        & (dfa.loc[dfa[0].isin(["21", "70", "30", "13", "60"])][2].astype(int) < 80)
    )
    | (
        (dfa[0].isin(["10"]))
        & (dfa[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
    )
]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [9]:
dfa.loc[dfa[0].isin(["21", "70", "30", "13", "60"]),2].head()

14        15
15        15
16        15
18    120910
19    120910
Name: 2, dtype: object

In [80]:
if df.loc[df[0]=="10"].empty:
    duration_min = int(df[4].at[0])
    date_col = 2
    time_col = 3
    add_day = 1
    date_col_name = "end_date"
    time_col_name = "end_time"
    typ = "sum"
else:
    date_col = 4
    time_col = 5
    duration_min = 0
    add_day = 0
    date_col_name = "departure_date"
    time_col_name = "departure_time"
    typ = "indv"
date_format = "%Y-%m-%d" # 2021-12-01
time_format = "%H:%M:%S.%f" # 00:00:00.00
    
date_length = len(df[date_col].at[0])

df[time_col] = df[time_col].astype(str)

df[time_col] = df[time_col].str.pad(width=7,side='right',fillchar="0")
df[time_col].loc[df[time_col].str[:2] == '24'] = ('0').zfill(7)

print("date_col:" + str(date_col))
print("time_col:" + str(time_col))

print(date_length)
print(df[date_col].head())

if date_length == 6:
    decade = int(df[date_col].at[0][:2])
    if decade < 50:
        century = str(date.today())[:2]
    else:
        century = int(str(date.today())[:2])-1
    df[date_col] = str(century) + df[date_col]
elif date_length == 8:
    pass
else:
    raise Exception("DATA Date length abnormal. length = "+str(date_length))

if typ == "indv":
    df[date_col] = pd.to_datetime(df[date_col], format="%Y%m%d").dt.strftime(date_format)
else:
    df[date_col] = df[date_col].apply(lambda x: pd.to_datetime(
        x, format="%Y%m%d").date() + timedelta(days=add_day)
        if x[time_col] in ['0'.zfill(7),'24'.ljust(7,'0')] 
        else pd.to_datetime(x, format="%Y%m%d").date())

df[time_col] = pd.to_datetime(df[time_col], format="%H%M%S%f").dt.strftime(time_format)
# df[time_col] = df[time_col].str[:11].astype(str)
print((df[date_col].astype(str)+df[time_col].astype(str)).head())


df['end_datetime'] = pd.to_datetime((df[date_col].astype(str)+df[time_col].str[:10].astype(str)), 
    format=date_format+time_format)

df['start_datetime'] = pd.to_datetime((df[date_col].astype(str)+df[time_col].str[:10].astype(str)), 
    format=date_format+time_format) - timedelta(minutes=duration_min)

df.rename(columns={
    date_col: date_col_name,
    time_col: time_col_name
},inplace = True)

date_col:2
time_col:3
6
0    120910
1    120910
2    120910
3    120910
4    120910
Name: 2, dtype: object
0    2012-09-1012:30:00.000000
1    2012-09-1012:30:00.000000
2    2012-09-1012:30:00.000000
3    2012-09-1012:30:00.000000
4    2012-09-1012:30:00.000000
dtype: object


In [81]:
if not dfa.empty:
    lanes = dfa.loc[
        dfa[0]=="L1"
        ].dropna(axis=1).drop_duplicates().reset_index(drop=True).copy()
    lanes = lanes.drop([17, 18, 19, 20, 21, 22, 23, 24, 25], axis=1, errors='ignore')
    if lanes.shape[1] == 5:
        lanes.rename(columns={
            1 : "lane_number",
            2 : "direction_code",
            3 : "lane_type_code",
            4 : "traffic_stream_number"
        },inplace = True)
    elif lanes.shape[1] == 11:
        lanes.rename(columns={
            1 : "lane_number",
            2 : "direction_code",
            3 : "lane_type_code",
            4 : "traffic_stream_number",
            5 : "traffic_stream_lane_position",
            6 : "reverse_direction_lane_number",
            7 : "vehicle_code",
            8 : "time_code",
            9 : "length_code",
            10 : "speed_code"
        },inplace = True)
    elif lanes.shape[1] == 17:
        lanes.rename(columns={
            1 : "lane_number",
            2 : "direction_code",
            3 : "lane_type_code",
            4 : "traffic_stream_number",
            5 : "traffic_stream_lane_position",
            6 : "reverse_direction_lane_number",
            7 : "vehicle_code",
            8 : "time_code",
            9 : "length_code",
            10 : "speed_code",
            11 : "occupancy_time_code",
            12 : "vehicle_following_code",
            13 : "trailer_code",
            14 : "axle_code",
            15 : "mass_code",
            16 : "tyre_type_code"
        },inplace = True)
    else:
        lanes.rename(columns={
            1 : "lane_number",
            2 : "direction_code",
            3 : "lane_type_code",
            4 : "traffic_stream_number"
        },inplace = True)
    lanes[lanes.select_dtypes(include=['object']).columns] = lanes[
        lanes.select_dtypes(include=['object']).columns].apply(
            pd.to_numeric, axis=1, errors='ignore')
    lanes["site_name"] = 'site_id'
else:
    pass


In [82]:
try:
    dir_1 = lanes["direction_code"].astype(int).min()
    dir_2 = lanes["direction_code"].astype(int).max()
except (TypeError,ValueError):
    dir_1 = 0
    dir_2 = 4
    
if df.loc[df[0]=="10"].empty:
    lane_col = 5
else:
    lane_col = 6


df['direction'] = df[lane_col].astype(int)
df['compass_heading'] = df[lane_col].astype(int)
df['direction_code'] = df[lane_col].astype(int)
df['direction'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_1])
    )] = 'P'
df['direction'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_2])
    )] = 'N'
df['compass_heading'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_1])
    )] = str(dir_1)
df['compass_heading'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_2])
    )] = str(dir_2)
df['direction_code'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_2])
    )] = str(dir_2)
df['direction_code'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_1])
    )] = str(dir_1)


In [21]:
data = df.loc[(df[0] == "21")].dropna(
    axis=1, how="all"
).reset_index(drop=True).copy()

if (data[1] == "0").any():
    ddf = data
    # ddf = data.iloc[:, 2:]

    ddf.rename(columns = {
        4 : "duration_min",
        5 : "lane_number",
        6 : "speedbin1",
        7 : "speedbin2",
        8 : "speedbin3",
        9 : "speedbin4",
        10 : "speedbin5",
        11 : "speedbin6",
        12 : "speedbin7",
        13 : "speedbin8",
        14 : "speedbin9",
        15 : "speedbin10",
        16 : "sum_of_heavy_vehicle_speeds",
        17 : "short_heavy_vehicles",
        18 : "medium_heavy_vehicles",
        19 : "long_heavy_vehicles",
        20 : "rear_to_rear_headway_shorter_than_2_seconds",
        21 : "rear_to_rear_headways_shorter_than_programmed_time"
        }, inplace=True)
    ddf["speedbin0"] = 0

elif (data[1] == "1").any():
    ddf = data
    # ddf = data.iloc[:, 3:]
    
    ddf.rename(columns = {
        4 : "duration_min",
        5 : "lane_number",
        6 : "speedbin0",
        7 : "speedbin1",
        8 : "speedbin2",
        9 : "speedbin3",
        10 : "speedbin4",
        11 : "speedbin5",
        12 : "speedbin6",
        13 : "speedbin7",
        14 : "speedbin8",
        15 : "speedbin9",
        16 : "speedbin10",
        17 : "sum_of_heavy_vehicle_speeds",
        18 : "short_heavy_vehicles",
        19 : "medium_heavy_vehicles",
        20 : "long_heavy_vehicles",
        21 : "rear_to_rear_headway_shorter_than_2_seconds",
        22 : "rear_to_rear_headways_shorter_than_programmed_time",
    },inplace=True)

ddf = ddf.fillna(0)

ddf["duration_min"] = ddf["duration_min"].astype(int)
ddf["lane_number"] = ddf["lane_number"].astype(int)
ddf["speedbin0"] = ddf["speedbin0"].astype(int)
ddf["speedbin1"] = ddf["speedbin1"].astype(int)
ddf["speedbin2"] = ddf["speedbin2"].astype(int)
ddf["speedbin3"] = ddf["speedbin3"].astype(int)
ddf["speedbin4"] = ddf["speedbin4"].astype(int)
ddf["speedbin5"] = ddf["speedbin5"].astype(int)
ddf["speedbin6"] = ddf["speedbin6"].astype(int)
ddf["speedbin7"] = ddf["speedbin7"].astype(int)
ddf["speedbin8"] = ddf["speedbin8"].astype(int)
ddf["speedbin9"] = ddf["speedbin9"].astype(int)
ddf["speedbin10"] = ddf["speedbin10"].astype(int)
ddf["sum_of_heavy_vehicle_speeds"] = ddf[
    "sum_of_heavy_vehicle_speeds"
].astype(int)
ddf["short_heavy_vehicles"] = ddf["short_heavy_vehicles"].astype(int)
ddf["medium_heavy_vehicles"] = ddf["medium_heavy_vehicles"].astype(int)
ddf["long_heavy_vehicles"] = ddf["long_heavy_vehicles"].astype(int)
ddf["rear_to_rear_headway_shorter_than_2_seconds"] = ddf[
    "rear_to_rear_headway_shorter_than_2_seconds"
].astype(int)
ddf["rear_to_rear_headways_shorter_than_programmed_time"] = ddf[
    "rear_to_rear_headways_shorter_than_programmed_time"
].astype(int)

ddf["total_heavy_vehicles_type21"] = (
    ddf["short_heavy_vehicles"]
    + ddf["medium_heavy_vehicles"]
    + ddf["long_heavy_vehicles"]
)
ddf["total_heavy_vehicles_type21"] = ddf[
    "total_heavy_vehicles_type21"
].astype(int)

ddf["total_light_vehicles_type21"] = (
    ddf["speedbin1"]
    + ddf["speedbin2"]
    + ddf["speedbin3"]
    + ddf["speedbin4"]
    + ddf["speedbin5"]
    + ddf["speedbin6"]
    + ddf["speedbin7"]
    + ddf["speedbin8"]
    + ddf["speedbin9"]
    + ddf["speedbin10"]
    - ddf["short_heavy_vehicles"]
    - ddf["medium_heavy_vehicles"]
    - ddf["long_heavy_vehicles"]
)
ddf["total_light_vehicles_type21"] = ddf[
    "total_light_vehicles_type21"
].astype(int)

ddf["total_vehicles_type21"] = (
    ddf["speedbin1"]
    + ddf["speedbin2"]
    + ddf["speedbin3"]
    + ddf["speedbin4"]
    + ddf["speedbin5"]
    + ddf["speedbin6"]
    + ddf["speedbin7"]
    + ddf["speedbin8"]
    + ddf["speedbin9"]
    + ddf["speedbin10"]
)
ddf["total_vehicles_type21"] = ddf["total_vehicles_type21"].astype(int)

# ddf["start_datetime"] = pd.to_datetime(str(ddf["start_datetime"]), 
# format="%Y-%m-%d %H:%M:%S")
try:
    ddf['year'] = ddf['start_datetime'].dt.year
except AttributeError:
    ddf['year'] = int(ddf['start_datetime'].str[:4][0])

ddf["site_id"] = "site_id"

# ddf = ddf.drop_duplicates()

In [83]:
data_df = df

In [129]:
if data_df is None:
    pass
else:
    data = data_df.loc[(data_df[0] == "60")].dropna(
        axis=1, how="all"
    ).reset_index(drop=True).copy()
    dfh = head_df.loc[(head_df[0] == "60")].dropna(
                axis=1, how="all"
            ).drop_duplicates().reset_index(drop=True).copy()
    dfh['error_bin'] = 0

    number_of_data_records = dfh.iloc[0,3]

    if data[1].isin(["0", "1", "2", "3", "4"]).any():
        ddf = data.iloc[:, 1:].reset_index(drop=True)
        ddf = pd.DataFrame(ddf).dropna(axis=1, how="all").reset_index(drop=True)

        ddf[ddf.select_dtypes(include=['object']).columns] = ddf[
            ddf.select_dtypes(include=['object']).columns
            ].apply(pd.to_numeric, axis = 1, errors='ignore')

        max_lanes = lanes['lane_number'].astype(int).max()

        ddf.columns = ddf.columns.astype(str)

        df3 = pd.DataFrame(columns=[
        'edit_code', 
        'start_datetime', 
        'end_date', 
        'end_time', 
        'duration_of_summary', 
        'lane_number', 
        'bin_number', 
        'number_of_vehicles', 
        'bin_boundary_length_cm', 
        'direction', 
        'compass_heading'])

        for i in range(6,int(number_of_data_records)+6):
            for lane_no in range(1, max_lanes+1):
                join_to_df3 = ddf.loc[ddf['5'].astype(int) == lane_no, [
                    '1', # edit_code
                    'start_datetime',
                    'end_date', # end_date
                    'end_time', # end_time
                    '4', # duration_of_summary
                    '5', # lane_number
                    str(i), # "number_of_vehicles"
                    'direction', 
                    'compass_heading'
                    ]]
                if str(int(i)-6) == "0":
                    bin_bound_col = 'error_bin'
                else:
                    bin_bound_col = int(i)-3
                join_to_df3['bin_number'] = str(i-6)
                join_to_df3['bin_boundary_length_cm'] = int(dfh[bin_bound_col][0])
                join_to_df3.rename(columns={
                    '1':"edit_code",
                    '4':"duration_of_summary",
                    '5':"lane_number",
                    str(i): "number_of_vehicles"
                    }, inplace=True)
                df3 = pd.concat([df3,join_to_df3], axis=0, ignore_index=True)
        df3 = df3.apply(pd.to_numeric, axis = 1, errors="ignore")
        df3['site_id'] = "site_id"
        df3['year'] = int(df3['start_datetime'].at[0].year)
    else:
        pass

In [130]:
df3['year'] = int(df3['start_datetime'].at[0].year)

In [146]:
int(lanes['lane_number'].astype(int).max())

7

In [125]:
for i in range(6,int(number_of_data_records)+6):
    if str(int(i)-6) == "0":
        x = 'error_bin'
    else:
        x = str(int(i)-3)
    print(i, " : ",str(int(i)-6), " : ", x)


6  :  0  :  error_bin
7  :  1  :  4
8  :  2  :  5
9  :  3  :  6
10  :  4  :  7
11  :  5  :  8
12  :  6  :  9
13  :  7  :  10
14  :  8  :  11
15  :  9  :  12
16  :  10  :  13
17  :  11  :  14
18  :  12  :  15
19  :  13  :  16
20  :  14  :  17
21  :  15  :  18
22  :  16  :  19
23  :  17  :  20
24  :  18  :  21
25  :  19  :  22


In [128]:
dfh[23]

KeyError: 23

In [60]:
df.loc[df[0].isin(["D1","D3"]), 0:4]

,0,1,2,3,4
3,D3,20000508,100000,20000508,234500
15,D3,20000509,0-5500,20000509,234500
27,D3,20000510,0-5500,20000510,234500
39,D3,20000511,0-5500,20000511,234500
51,D3,20000512,0-5500,20000512,234500
63,D3,20000513,0-5500,20000513,234500
75,D3,20000514,0-5500,20000514,234500
87,D3,20000515,0-5500,20000515,234500
99,D3,20000516,0-5500,20000516,234500
111,D3,20000517,0-5500,20000517,234500


In [54]:
# def header(self, df: pd.DataFrame) -> pd.DataFrame:
head_df = head_df.reset_index(drop=True)
st_nd = head_df.loc[head_df[0].isin(["D1","D3"]), 0:4].reset_index(drop=True).copy()

# adds century to year if it is not there
st_nd[1] = st_nd[1].apply(lambda x: str(date.today())[:2] + x if len(x)==6 else x)
st_nd[3] = st_nd[3].apply(lambda x: str(date.today())[:2] + x if len(x)==6 else x)

st_nd[1] = st_nd[1].str.pad(width=8, side='right', fillchar="0")
st_nd[3] = st_nd[3].str.pad(width=8, side='right', fillchar="0")

# index 2 and 4 are time, this makes the time uniform length
st_nd[2] = st_nd[2].str.pad(width=7,side='right',fillchar="0")
st_nd[4] = st_nd[4].str.pad(width=7,side='right',fillchar="0")

# this filters for time = 24H00m and makes it zero hour
st_nd[2].loc[st_nd[2].str[:2] == '24'] = ('0').zfill(7)
st_nd[4].loc[st_nd[4].str[:2] == '24'] = ('0').zfill(7)

# adds a day if the hour is zero hour and changes string to dtetime.date
try:
    st_nd[1] = st_nd[1].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
    if x[2] in ['0'.zfill(7),'24'.zfill(7)] else pd.to_datetime(x, format="%Y%m%d").date())
    st_nd[3] = st_nd[3].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
    if x[4] in ['0'.zfill(7),'24'.zfill(7)] else pd.to_datetime(x, format="%Y%m%d").date())
# except ValueError:
#     st_nd[1] = st_nd[1].apply(lambda x: pd.to_datetime(x[:8], format="%Y%m%d").date() + timedelta(days=1)
#     if x[2] in ['0'.zfill(7),'24'.zfill(7)] else pd.to_datetime(x[:8], format="%Y%m%d").date())
#     st_nd[3] = st_nd[3].apply(lambda x: pd.to_datetime(x[:8], format="%Y%m%d").date() + timedelta(days=1)
#     if x[4] in ['0'.zfill(7),'24'.zfill(7)] else pd.to_datetime(x[:8], format="%Y%m%d").date())
except:
    pass

# changes time string into datetime.time
try:
    st_nd[2] = st_nd[2].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())
    st_nd[4] = st_nd[4].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())
except ValueError:
    st_nd[2] = pd.to_datetime("0".zfill(7), format="%H%M%S%f").strftime("%H:%M:%S")
    st_nd[4] = pd.to_datetime("0".zfill(7), format="%H%M%S%f").strftime("%H:%M:%S")
except:
    pass

# creates start_datetime and end_datetime
try:
    st_nd["start_datetime"] = pd.to_datetime((st_nd[1].astype(str)+st_nd[2].astype(str)), 
        format='%Y-%m-%d%H:%M:%S')
    st_nd["end_datetime"] = pd.to_datetime((st_nd[3].astype(str)+st_nd[4].astype(str)), 
        format='%Y-%m-%d%H:%M:%S')
except Exception:
    pass


st_nd = st_nd.iloc[:,1:].drop_duplicates()

headers = pd.DataFrame()
try:
    headers['start_datetime'] = st_nd.groupby(st_nd['start_datetime'].dt.year).min()['start_datetime']
    headers['end_datetime'] = st_nd.groupby(st_nd['end_datetime'].dt.year).max()['end_datetime']
except:
    pass

headers['site_id'] = 'site_id'
headers['document_url'] = 'document_url'

headers["header_id"] = ""
headers["header_id"] = headers["header_id"].apply(
            lambda x: str(uuid.uuid4()))

headers['year'] = headers['start_datetime'].dt.year

headers["number_of_lanes"] = int(head_df.loc[head_df[0] == "L0", 2].drop_duplicates().reset_index(drop=True)[0])

station_name = head_df.loc[head_df[0].isin(["S0"]), 3:].reset_index(drop=True).drop_duplicates().dropna(axis=1)
headers["station_name"] = station_name[station_name.columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

t21 = head_df.loc[head_df[0]=="21"].dropna(axis=1).drop_duplicates().reset_index().copy()
t21 = t21.iloc[:,(t21.shape[1])-9:].astype(int)
t21.columns = range(t21.columns.size)
t21.rename(columns={
    0:'speedbin1',
    1:'speedbin2',
    2:'speedbin3',
    3:'speedbin4',
    4:'speedbin5',
    5:'speedbin6',
    6:'speedbin7',
    7:'speedbin8',
    8:'speedbin9'}
    ,inplace = True)

headers = pd.concat([headers,t21],ignore_index=True,axis=0).bfill().ffill().drop_duplicates()

try:
    headers["type_21_count_interval_minutes"] = int(head_df.loc[head_df[0]=="21",1].drop_duplicates().reset_index(drop=True)[0])
    headers["type_30_summary_interval_minutes"] = int(head_df.loc[head_df[0]=="21",1].drop_duplicates().reset_index(drop=True)[0])
    headers["type_70_summary_interval_minutes"] = int(head_df.loc[head_df[0]=="21",1].drop_duplicates().reset_index(drop=True)[0])
except KeyError:
    pass
try:
    headers["type_30_summary_interval_minutes"] = int(head_df.loc[head_df[0]=="30",1].drop_duplicates().reset_index(drop=True)[0])
    headers["type_21_count_interval_minutes"] = int(head_df.loc[head_df[0]=="30",1].drop_duplicates().reset_index(drop=True)[0])
    headers["type_70_summary_interval_minutes"] = int(head_df.loc[head_df[0]=="30",1].drop_duplicates().reset_index(drop=True)[0])
    headers["type_30_vehicle_classification_scheme"] = int(head_df.loc[head_df[0] == "30", 3].drop_duplicates().reset_index(drop=True)[0])
except KeyError:
    pass
try:
    headers["type_30_summary_interval_minutes"] = int(head_df.loc[head_df[0]=="70",1].drop_duplicates().reset_index(drop=True)[0])
    headers["type_21_count_interval_minutes"] = int(head_df.loc[head_df[0]=="70",1].drop_duplicates().reset_index(drop=True)[0])
    headers["type_70_summary_interval_minutes"] = int(head_df.loc[head_df[0]=="70",1].drop_duplicates().reset_index(drop=True)[0])
except KeyError:
    pass

try:
    headers['dir1_id'] = int(head_df[head_df[0]=="L1"].dropna(axis=1).drop_duplicates().reset_index(drop=True)[2].min())
    headers['dir2_id'] = int(head_df[head_df[0]=="L1"].dropna(axis=1).drop_duplicates().reset_index(drop=True)[2].max())
except (KeyError, ValueError):
    headers['dir1_id'] = 0
    headers['dir2_id'] = 4

try:
    headers["type_70_vehicle_classification_scheme"] = int(head_df.loc[
        head_df[0] == "70", 2].drop_duplicates().reset_index(drop=True)[0])
    headers["type_70_maximum_gap_milliseconds"] = int(head_df.loc[
        head_df[0] == "70", 3].drop_duplicates().reset_index(drop=True)[0])
    headers["type_70_maximum_differential_speed"] = int(head_df.loc[
        head_df[0] == "70", 4].drop_duplicates().reset_index(drop=True)[0])
    headers["type_70_error_bin_code"] = int(head_df.loc[
        head_df[0] == "70", 5].drop_duplicates().reset_index(drop=True)[0])
except KeyError:
    pass

headers = headers.reset_index(drop=True)

m = headers.select_dtypes(np.number)
headers[m.columns] = m.round().astype('Int32')

# return headers

In [55]:
headers

,start_datetime,end_datetime,site_id,document_url,header_id,year,number_of_lanes,station_name,speedbin1,speedbin2,speedbin3,speedbin4,speedbin5,speedbin6,speedbin7,speedbin8,speedbin9,type_21_count_interval_minutes,type_30_summary_interval_minutes,type_70_summary_interval_minutes,dir1_id,dir2_id
0,2011-10-05 14:45:00,2011-10-13 11:15:00,site_id,document_url,bb7e011f-773a-4e5d-9821-2e025fbceb4b,2011,2,<NA>,30,40,50,60,70,80,90,100,110,15,15,15,0,0


In [56]:
def type_21(df) -> pd.DataFrame: # deals with type 21
    if df is None:
        pass
    else:
        data = df.loc[(df[0] == "21")].dropna(
            axis=1, how="all"
        ).reset_index(drop=True).copy()

        if (data[1] == "0").any():
            ddf = data
            # ddf = data.iloc[:, 2:]

            ddf.rename(columns = {
                4 : "duration_min",
                5 : "lane_number",
                6 : "speedbin1",
                7 : "speedbin2",
                8 : "speedbin3",
                9 : "speedbin4",
                10 : "speedbin5",
                11 : "speedbin6",
                12 : "speedbin7",
                13 : "speedbin8",
                14 : "speedbin9",
                15 : "speedbin10",
                16 : "sum_of_heavy_vehicle_speeds",
                17 : "short_heavy_vehicles",
                18 : "medium_heavy_vehicles",
                19 : "long_heavy_vehicles",
                20 : "rear_to_rear_headway_shorter_than_2_seconds",
                21 : "rear_to_rear_headways_shorter_than_programmed_time"
                }, inplace=True)
            ddf["speedbin0"] = 0

        elif (data[1] == "1").any():
            ddf = data
            # ddf = data.iloc[:, 3:]
            
            ddf.rename(columns = {
                4 : "duration_min",
                5 : "lane_number",
                6 : "speedbin0",
                7 : "speedbin1",
                8 : "speedbin2",
                9 : "speedbin3",
                10 : "speedbin4",
                11 : "speedbin5",
                12 : "speedbin6",
                13 : "speedbin7",
                14 : "speedbin8",
                15 : "speedbin9",
                16 : "speedbin10",
                17 : "sum_of_heavy_vehicle_speeds",
                18 : "short_heavy_vehicles",
                19 : "medium_heavy_vehicles",
                20 : "long_heavy_vehicles",
                21 : "rear_to_rear_headway_shorter_than_2_seconds",
                22 : "rear_to_rear_headways_shorter_than_programmed_time",
            },inplace=True)
        
        ddf = ddf.fillna(0)

        ddf["duration_min"] = ddf["duration_min"].astype(int)
        ddf["lane_number"] = ddf["lane_number"].astype(int)
        ddf["speedbin0"] = ddf["speedbin0"].astype(int)
        ddf["speedbin1"] = ddf["speedbin1"].astype(int)
        ddf["speedbin2"] = ddf["speedbin2"].astype(int)
        ddf["speedbin3"] = ddf["speedbin3"].astype(int)
        ddf["speedbin4"] = ddf["speedbin4"].astype(int)
        ddf["speedbin5"] = ddf["speedbin5"].astype(int)
        ddf["speedbin6"] = ddf["speedbin6"].astype(int)
        ddf["speedbin7"] = ddf["speedbin7"].astype(int)
        ddf["speedbin8"] = ddf["speedbin8"].astype(int)
        ddf["speedbin9"] = ddf["speedbin9"].astype(int)
        ddf["speedbin10"] = ddf["speedbin10"].astype(int)
        ddf["sum_of_heavy_vehicle_speeds"] = ddf[
            "sum_of_heavy_vehicle_speeds"
        ].astype(int)
        ddf["short_heavy_vehicles"] = ddf["short_heavy_vehicles"].astype(int)
        ddf["medium_heavy_vehicles"] = ddf["medium_heavy_vehicles"].astype(int)
        ddf["long_heavy_vehicles"] = ddf["long_heavy_vehicles"].astype(int)
        ddf["rear_to_rear_headway_shorter_than_2_seconds"] = ddf[
            "rear_to_rear_headway_shorter_than_2_seconds"
        ].astype(int)
        ddf["rear_to_rear_headways_shorter_than_programmed_time"] = ddf[
            "rear_to_rear_headways_shorter_than_programmed_time"
        ].astype(int)

        ddf["total_heavy_vehicles_type21"] = (
            ddf["short_heavy_vehicles"]
            + ddf["medium_heavy_vehicles"]
            + ddf["long_heavy_vehicles"]
        )
        ddf["total_heavy_vehicles_type21"] = ddf[
            "total_heavy_vehicles_type21"
        ].astype(int)

        ddf["total_light_vehicles_type21"] = (
            ddf["speedbin1"]
            + ddf["speedbin2"]
            + ddf["speedbin3"]
            + ddf["speedbin4"]
            + ddf["speedbin5"]
            + ddf["speedbin6"]
            + ddf["speedbin7"]
            + ddf["speedbin8"]
            + ddf["speedbin9"]
            + ddf["speedbin10"]
            - ddf["short_heavy_vehicles"]
            - ddf["medium_heavy_vehicles"]
            - ddf["long_heavy_vehicles"]
        )
        ddf["total_light_vehicles_type21"] = ddf[
            "total_light_vehicles_type21"
        ].astype(int)

        ddf["total_vehicles_type21"] = (
            ddf["speedbin1"]
            + ddf["speedbin2"]
            + ddf["speedbin3"]
            + ddf["speedbin4"]
            + ddf["speedbin5"]
            + ddf["speedbin6"]
            + ddf["speedbin7"]
            + ddf["speedbin8"]
            + ddf["speedbin9"]
            + ddf["speedbin10"]
        )
        ddf["total_vehicles_type21"] = ddf["total_vehicles_type21"].astype(int)

        try:
            ddf['year'] = ddf['start_datetime'].dt.year
        except AttributeError:
            ddf['year'] = int(ddf['start_datetime'].str[:4][0])

        ddf["site_id"] = "site_id"

        return ddf

In [62]:
t21_data = type_21(df)

In [64]:
def header_calcs(header: pd.DataFrame, data: pd.DataFrame, type: int) -> pd.DataFrame:
    try:
        speed_limit_qry = f"select max_speed from trafc.countstation where tcname = '{data['site_id'].iloc[0]}' ;"
        speed_limit = pd.read_sql_query(speed_limit_qry,config.ENGINE).reset_index(drop=True)
        try:
            speed_limit = speed_limit['max_speed'].iloc[0]
        except IndexError:
            speed_limit = 60
        data = data.fillna(0, axis=0)
        if type == 21:
            try:
                header['adt_total'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D')]).sum().mean().round().astype(int)
                header['adt_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P']]).sum().mean())
                header['adt_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N']]).sum().mean())

                header['adtt_total'] = data['total_heavy_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D')]).sum().mean().round().astype(int)
                header['adtt_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P']]).sum().mean())
                header['adtt_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N']]).sum().mean())

                header['total_vehicles'] = data['total_vehicles_type21'].sum()
                header['total_vehicles_positive_direction'] = data['total_vehicles_type21'].groupby(data['direction'].loc[data['direction'] == 'P']).sum()[0]
                header['total_vehicles_negative_direction'] = data['total_vehicles_type21'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[0]

                header['total_heavy_vehicles'] = data['total_heavy_vehicles_type21'].sum()
                header['total_heavy_negative_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['truck_split_negative_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / data['total_heavy_vehicles_type21'].sum()
                header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / data['total_heavy_vehicles_type21'].sum()

                header['total_light_vehicles'] = data['total_light_vehicles_type21'].sum()
                header['total_light_positive_direction'] = data['total_light_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['total_light_negative_direction'] = data['total_light_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]

                header['short_heavy_vehicles'] = data['short_heavy_vehicles'].sum()
                header['short_heavy_positive_direction'] = data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['short_heavy_negative_direction'] = data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]

                header['Medium_heavy_vehicles'] = data['medium_heavy_vehicles'].sum()
                header['Medium_heavy_negative_direction'] = data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                header['Medium_heavy_positive_direction'] = data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]

                header['long_heavy_vehicles'] = data['long_heavy_vehicles'].sum()
                header['long_heavy_positive_direction'] = data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['long_heavy_negative_direction'] = data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]

                header['vehicles_with_rear_to_rear_headway_less_than_2sec_positive_dire'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['vehicles_with_rear_to_rear_headway_less_than_2sec_negative_dire'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                header['vehicles_with_rear_to_rear_headway_less_than_2sec_total'] = data['rear_to_rear_headway_shorter_than_2_seconds'].sum()
            
                header['type_21_count_interval_minutes'] = data['duration_min'].mean()

                header['highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P']]).sum().max())
                header['highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N']]).sum().max())
                header['highest_volume_per_hour_total'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H')]).sum().max()

                header['15th_highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P']]).sum().quantile(q=0.15,  interpolation='linear'))
                header['15th_highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N']]).sum().quantile(q=0.15,  interpolation='linear'))
                header['15th_highest_volume_per_hour_total'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H')]).sum().quantile(q=0.15, interpolation='linear'))
                
                header['30th_highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P']]).sum().quantile(q=0.3,  interpolation='linear'))
                header['30th_highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N']]).sum().quantile(q=0.3, interpolation='linear'))
                header['30th_highest_volume_per_hour_total'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H')]).sum().quantile(q=0.3, interpolation='linear'))

                # header['average_speed_positive_direction'] = 
                # header['average_speed_negative_direction'] = 
                try:
                    header['average_speed'] = (int((
                    (header['speedbin1'] * data['speedbin1'].sum()) +
                    (header['speedbin2'] * data['speedbin2'].sum()) +
                    (header['speedbin3'] * data['speedbin3'].sum()) +
                    (header['speedbin4'] * data['speedbin4'].sum()) +
                    (header['speedbin5'] * data['speedbin5'].sum()) +
                    (header['speedbin6'] * data['speedbin6'].sum()) +
                    (header['speedbin7'] * data['speedbin7'].sum()) +
                    (header['speedbin8'] * data['speedbin8'].sum()) +
                    (header['speedbin9'] * data['speedbin9'].sum()) 
                    ))
                    / data['sum_of_heavy_vehicle_speeds'].astype(int).sum()
                    )
                except TypeError:
                    header['average_speed'] = (((
                    (header['speedbin1'] * data['speedbin1'].astype(int).sum()) +
                    (header['speedbin2'] * data['speedbin2'].astype(int).sum()) +
                    (header['speedbin3'] * data['speedbin3'].astype(int).sum()) +
                    (header['speedbin4'] * data['speedbin4'].astype(int).sum()) +
                    (header['speedbin5'] * data['speedbin5'].astype(int).sum()) +
                    (header['speedbin6'] * data['speedbin6'].astype(int).sum()) +
                    (header['speedbin7'] * data['speedbin7'].astype(int).sum()) +
                    (header['speedbin8'] * data['speedbin8'].astype(int).sum()) +
                    (header['speedbin9'] * data['speedbin9'].astype(int).sum()) 
                    ))
                    / data['sum_of_heavy_vehicle_speeds'].astype(int).sum()
                    )
                # header['average_speed_light_vehicles_positive_direction'] = 
                # header['average_speed_light_vehicles_negative_direction'] = 
                header['average_speed_light_vehicles'] = ((
                    (header['speedbin1'] * data['speedbin1'].sum()) +
                    (header['speedbin2'] * data['speedbin2'].sum()) +
                    (header['speedbin3'] * data['speedbin3'].sum()) +
                    (header['speedbin4'] * data['speedbin4'].sum()) +
                    (header['speedbin5'] * data['speedbin5'].sum()) +
                    (header['speedbin6'] * data['speedbin6'].sum()) +
                    (header['speedbin7'] * data['speedbin7'].sum()) +
                    (header['speedbin8'] * data['speedbin8'].sum()) +
                    (header['speedbin9'] * data['speedbin9'].sum()) -
                    data['sum_of_heavy_vehicle_speeds'].sum()
                    )
                    / data['sum_of_heavy_vehicle_speeds'].sum()
                    )
                
                # header['average_speed_heavy_vehicles_positive_direction'] = 
                # header['average_speed_heavy_vehicles_negative_direction'] = 
                # header['average_speed_heavy_vehicles'] = 
                
                try:
                    header['truck_split_positive_direction'] = (str(
                        round(data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]*100)) 
                    + ' : ' +
                    str(
                        round(data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]*100)) 
                    + ' : ' +
                    str(
                        round(data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]*100))
                    )
                except ValueError:
                    pass
                
                try:
                    header['truck_split_negative_direction'] = (str(
                        round(data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()*100)) 
                        + ' : ' +
                    str(
                        round(data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]*100)) 
                    + ' : ' +
                    str(
                        round(data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]*100))
                    )
                except ValueError:
                    pass
                try:
                    header['truck_split_total'] = (str(
                        round(data['short_heavy_vehicles'].sum() / 
                        data['total_heavy_vehicles_type21'].sum()*100)) 
                        + ' : ' +
                    str(
                        round(data['medium_heavy_vehicles'].sum() / 
                        data['total_heavy_vehicles_type21'].sum()*100)) 
                    + ' : ' +
                    str(
                        round(data['long_heavy_vehicles'].sum() / 
                        data['total_heavy_vehicles_type21'].sum()*100))
                    )
                except ValueError:
                    pass

            except KeyError:
                pass
            try:
                header["type_21_count_interval_minutes"] = header["type_21_count_interval_minutes"].round().astype(int)
            except (KeyError, pd.errors.IntCastingNaNError):
                pass

            return header
        
        elif type == 30:
            try:
                if header['adt_total'].isnull().all():
                    header['adt_total'] = data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D')]).sum().mean().astype(int)
                    header['adt_positive_direction'] = round(data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P']]).sum().mean())
                    header['adt_negative_direction'] = round(data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N']]).sum().mean())
                else:
                    pass

                if header['adtt_total'].isnull().all():
                    header['adtt_total'] = data['total_heavy_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D')]).sum().mean().astype(int)
                    header['adtt_positive_direction'] = round(data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P']]).sum().mean())
                    header['adtt_negative_direction'] = round(data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N']]).sum().mean())
                else:
                    pass

                if header['total_vehicles'].isnull().all():
                    header['total_vehicles'] = data['total_vehicles_type30'].sum()
                    header['total_vehicles_positive_direction'] = data['total_vehicles_type30'].groupby(data['direction'].loc[data['direction'] == 'P']).sum()[0]
                    header['total_vehicles_negative_direction'] = data['total_vehicles_type30'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[0]
                else:
                    pass
                
                if header['total_heavy_vehicles'].isnull().all():
                    header['total_heavy_vehicles'] = data['total_heavy_vehicles_type21'].sum()
                    header['total_heavy_negative_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                    header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                    header['truck_split_negative_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / data['total_heavy_vehicles_type21'].sum()
                    header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / data['total_heavy_vehicles_type21'].sum()
                else:
                    pass

                if header['total_light_vehicles'].isnull().all():
                    header['total_light_vehicles'] = data['total_light_vehicles_type30'].sum()
                    header['total_light_positive_direction'] = data['total_light_vehicles_type30'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                    header['total_light_negative_direction'] = data['total_light_vehicles_type30'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                else:
                    pass
            except KeyError:
                pass
                
            try:
                header['type_30_vehicle_classification_scheme'] = header['type_30_vehicle_classification_scheme'].round().astype(int)
            except (KeyError, pd.errors.IntCastingNaNError):
                pass

            return header

        elif type == 70:

            try:
                header['type_70_maximum_gap_milliseconds'] = header['type_70_maximum_gap_milliseconds'].round().astype(int)
            except (KeyError, pd.errors.IntCastingNaNError):
                pass
            
            return header
        
        elif type == 10:
            header['total_light_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_light_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_light_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']<=1].count()[0].round().astype(int)
            header['total_heavy_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_heavy_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_heavy_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']>1].count()[0].round().astype(int)
            header['total_short_heavy_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_short_heavy_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_short_heavy_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0].round().astype(int)
            header['total_medium_heavy_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_medium_heavy_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_medium_heavy_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0].round().astype(int)
            header['total_long_heavy_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_long_heavy_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_long_heavy_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0].round().astype(int)
            header['total_vehicles_negative_direction'] = data.loc[data['direction']=='N'].count()[0].round().astype(int)
            header['total_vehicles_positive_direction'] = data.loc[data['direction']=='P'].count()[0].round().astype(int)
            header['total_vehicles'] = data.count()[0].round().astype(int)
            header['average_speed_negative_direction'] = data.loc[data['direction']=='N']['vehicle_speed'].mean().round(2)
            header['average_speed_positive_direction'] = data.loc[data['direction']=='P']['vehicle_speed'].mean().round(2)
            header['average_speed'] = data['vehicle_speed'].mean().round(2)
            header['average_speed_light_vehicles_negative_direction'] = data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='N')].mean().round(2)
            header['average_speed_light_vehicles_positive_direction'] = data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='P')].mean().round(2)
            header['average_speed_light_vehicles'] = data['vehicle_speed'].loc[data['vehicle_class_code_secondary_scheme']<=1].mean().round(2)
            header['average_speed_heavy_vehicles_negative_direction'] = data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].mean().round(2)
            header['average_speed_heavy_vehicles_positive_direction'] = data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].mean().round(2)
            header['average_speed_heavy_vehicles'] = data['vehicle_speed'].loc[data['vehicle_class_code_secondary_scheme']>1].mean().round(2)
            header['truck_split_negative_direction'] = {str((((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int))}
            header['truck_split_positive_direction'] = {str((((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int))}
            header['truck_split_total'] = {str((((data.loc[data['vehicle_class_code_secondary_scheme']==2].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[data['vehicle_class_code_secondary_scheme']==3].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[data['vehicle_class_code_secondary_scheme']==4].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int))}
            header['estimated_axles_per_truck_negative_direction'] = ((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]*2+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]*5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0]*7)/(data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0])).round(2)
            header['estimated_axles_per_truck_positive_direction'] = ((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]*2+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]*5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0]*7)/(data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0])).round(2)
            header['estimated_axles_per_truck_total'] = ((data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]*2+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]*5+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0]*7)/(data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0])).round(2)
            header['percentage_speeding_positive_direction'] = ((data.loc[(data['vehicle_speed']>speed_limit)&(data['direction']=='P')].count()[0]/data.loc[data['direction'=='P']].count()[0])*100).round(2)
            header['percentage_speeding_negative_direction'] = ((data.loc[(data['vehicle_speed']>speed_limit)&(data['direction']=='N')].count()[0]/data.loc[data['direction'=='N']].count()[0])*100).round(2)
            header['percentage_speeding_total'] = ((data.loc[data['vehicle_speed']>speed_limit].count()[0]/data.count()[0])*100).round(2)
            header['vehicles_with_rear_to_rear_headway_less_than_2sec_negative_dire'] = data.loc[(data['vehicle_following_code']==2)&data['direction']=='N'].count()[0]
            header['vehicles_with_rear_to_rear_headway_less_than_2sec_positive_dire'] = data.loc[(data['vehicle_following_code']==2)&data['direction']=='P'].count()[0]
            header['vehicles_with_rear_to_rear_headway_less_than_2sec_total'] = data.loc[data['vehicle_following_code']==2].count()[0]
            header['estimated_e80_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]*0.6+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]*2.5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0]*2.1
            header['estimated_e80_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]*0.6+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]*2.5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0]*2.1
            header['estimated_e80_on_road'] = data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]*0.6+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]*2.5+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0]*2.1
            header['adt_negative_direction'] = data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adt_positive_direction'] = data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adt_total'] = data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adtt_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adtt_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adtt_total'] = data.loc[data['vehicle_class_code_secondary_scheme']>1].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['highest_volume_per_hour_negative_direction'] = data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]
            header['highest_volume_per_hour_positive_direction'] = data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]
            header['highest_volume_per_hour_total'] = data.groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]
            header["15th_highest_volume_per_hour_negative_direction"] = round(data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].round().astype(int))
            header["15th_highest_volume_per_hour_positive_direction"] = round(data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].round().astype(int))
            header["15th_highest_volume_per_hour_total"] = data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].round().astype(int)
            header["30th_highest_volume_per_hour_negative_direction"] = round(data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].round().astype(int))
            header["30th_highest_volume_per_hour_positive_direction"] = round(data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].round().astype(int))
            header["30th_highest_volume_per_hour_total"] = data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].round().astype(int)
            header["15th_percentile_speed_negative_direction"] = data.loc[data['direction']=='N']['vehicle_speed'].quantile(0.15).round(2)
            header["15th_percentile_speed_positive_direction"] = data.loc[data['direction']=='P']['vehicle_speed'].quantile(0.15).round(2)
            header["15th_percentile_speed_total"] = data['vehicle_speed'].quantile(0.15).round(2)
            header["85th_percentile_speed_negative_direction"] = data.loc[data['direction']=='N']['vehicle_speed'].quantile(0.85).round(2)
            header["85th_percentile_speed_positive_direction"] = data.loc[data['direction']=='P']['vehicle_speed'].quantile(0.85).round(2)
            header["85th_percentile_speed_total"] = data['vehicle_speed'].quantile(0.85).round(2)
            header['avg_weekday_traffic'] = data.groupby(pd.Grouper(key='start_datetime',freq='B')).count().mean()[0].round().astype(int)
            header['number_of_days_counted'] = data.groupby([data['start_datetime'].dt.to_period('D')]).count().count()[0]
            header['duration_hours'] = data.groupby([data['start_datetime'].dt.to_period('H')]).count().count()[0]

            return header

        elif type == 60:
            
            return header
        else:
            return header
    except IndexError:
        return header



In [69]:
t21_data['total_vehicles_type21'].groupby([t21_data['start_datetime'].dt.to_period('D')]).sum()

start_datetime
2011-10-04       5
2011-10-05    1903
2011-10-06    5435
2011-10-07    6079
2011-10-08    3708
2011-10-09    2463
2011-10-10    5433
2011-10-11    5869
2011-10-12    5750
2011-10-13    2300
Freq: D, Name: total_vehicles_type21, dtype: int32

In [76]:
t21_data['total_vehicles_type21'].groupby(t21_data['direction'].loc[t21_data['direction'] == 'P']).sum()[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [72]:
round(t21_data['total_vehicles_type21'].groupby([t21_data['start_datetime'].dt.to_period('D'), t21_data['direction'].loc[t21_data['direction'] == 'P']]).sum().mean()) or 0

ValueError: cannot convert float NaN to integer